In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
from concurrent.futures import ThreadPoolExecutor
import urllib.parse
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/69.0.3497.100 ' +
                         'Safari/537.36' +
                         'OPR/56.0.3051.99'}

Функции для получения списка id книг автора по его имени

In [2]:
def get_author_url(name):
    name_uni = urllib.parse.quote(name, encoding='windows-1251')
    return f'https://www.chitai-gorod.ru/search/result.php?q={name_uni}&type=author'


def get_id_list(author_url, driver):
        driver.get(author_url)

        #get expected number of ids
        el_n_cards = driver.find_elements_by_class_name("count-result__value")
        n_cards = int(el_n_cards[0].text)
        
        #scroll the webpage to the botton 
        #until we got number equal to the expected one
        while len(driver.find_elements_by_class_name("product-card")) != n_cards:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            #try to click button if available
            #catch exception if it executed automatically
            #or not in frame
            elem = driver.find_elements_by_css_selector(".js__show-more-cards")
            if elem:
                try:
                    elem[0].click()
                except:
                    pass

        return [int(item.get_attribute("data-product"))
               for item in driver.find_elements_by_class_name("product-card")]

Функции для парсинга страницы с книгой по id её карточки

In [3]:
def get_table_dict(table):
    keys = []
    for item in table.findAll('div', class_="product-prop__title"):
        keys.append(item.text.lstrip().rstrip())
    values = []
    for item in table.findAll('div', class_="product-prop__value"):
        values.append(item.text.lstrip().rstrip())
        
    return dict(zip(keys, values))


def table_dict_preprocess(table_dict):
    for key, _ in table_dict.items():
        if key in {'Год издания', 'Кол-во страниц', 'Тираж'}:
            table_dict[key] = int(re.findall(r'\d+', table_dict[key])[0])
        if key == 'ID товара:':
            #web-developers misprint fix
            table_dict['ID товара'] = int(table_dict[key])
            table_dict.pop(key)
    return table_dict


def extract_price(string):
    return int(re.findall(r'\d+', string)[0])


def extract_rating(string):
    return float(re.findall(r'\d+', string)[0]), int(re.findall(r'\d+', string)[1])

In [4]:
def get_book_info(book_id):
    url = f'https://www.chitai-gorod.ru/catalog/book/{book_id}/'
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, 'html.parser')
    
    info = {}
    info['ID карточки'] = book_id
    nameSoup = soup.find('h1', class_='product__title js-analytic-product-title')
    info['Название'] = nameSoup.text.lstrip().rstrip()
    authorSoup = soup.find('a', class_='link product__author')
    info['Автор'] = authorSoup.text.lstrip().rstrip()
    
    rating = soup.select_one('.rating').text.lstrip().rstrip()
    info['Рейтинг'], info['Голоса'] = extract_rating(rating)
    
    price_el = soup.find('div', class_="product__price")
    if price_el is not None:
        price = price_el.text.lstrip().rstrip()
        info['Цена'] = extract_price(price)
    
    table = soup.find('div', class_="product__props")
    table_dict = get_table_dict(table)
    info.update(table_dict_preprocess(table_dict))
    
    image_el = soup.select_one('.product__image')
    if image_el is not None:
        info['Обложка'] = image_el.find('img')['src']
    
    return info

Маппер для параллельного обкачивания информации о книгах

In [5]:
def pool_mapper(func, data, max_workers=32):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
            return executor.map(func, data) 

Функция для получения списка данных о всех книгах автора (авторов) по его (их) имени

In [6]:
def get_author_library(author):
    url = get_author_url(author)
    
    with webdriver.Firefox() as driver:
        id_data = get_id_list(url, driver)
    
    return [info for info in pool_mapper(get_book_info, id_data)]


def get_authors_library(authors_list):
    authors_url = [get_author_url(author) for author in authors_list]

    with webdriver.Firefox() as driver:
        id_list = [item for url in authors_url for item in get_id_list(url, driver)]
    
    return [info for info in pool_mapper(get_book_info, id_list)]

Функция для перевода библиотеки в формат pandas.DataFrame и её записи в файл формата .csv

In [7]:
def write_library_to_csv(library, path_name):
    df = pd.DataFrame(library)
    df.sort_values(by='ID карточки', inplace=True)

    with open(path_name, mode='w', encoding='utf-8') as f_csv:
        df.to_csv(f_csv, index=False)
        
    return df

Выполнение кода для предложенного набора авторов

In [8]:
%%time


authors_list = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
library = get_authors_library(authors_list)
df = write_library_to_csv(library, './hw_4.csv')

CPU times: user 1min 8s, sys: 2.82 s, total: 1min 11s
Wall time: 4min 21s


In [9]:
df.tail(10)

,ID карточки,ID товара,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена
2,1059170,2659485,9785171089733,Фрай М.,16+,2018,20,АСТ,448.0,Мертвый ноль,https://img-gorod.ru/upload/iblock/aec/aec2cfa...,NaN,NaN,4.0,Сновидения Ехо,Твердая бумажная,30000.0,20.5 x 13 x 2.5,NaN,375.0
252,1062116,2662062,9785699973781,Емец Д.,12+,2018,0,Эксмо,416.0,Стеклянный страж,https://img-gorod.ru/upload/iblock/b05/b05cc44...,NaN,NaN,0.0,Мефодий Буслаев. Легендарное детское фэнтези,Твердая бумажная,2000.0,21.8 x 14.2 x 2.4,NaN,327.0
251,1064110,2664251,9785040916658,Емец Д.,6+,2018,1,Эксмо,272.0,Золото скифов,https://img-gorod.ru/upload/iblock/793/793a002...,NaN,NaN,5.0,NaN,Твердая бумажная,5000.0,21.8 x 14.7 x 1.8,NaN,285.0
156,1066041,2665312,9785001113539,Хантер Э.,6+,2018,3,Абрис,320.0,Сны и видения Бабочки. Целительница,https://img-gorod.ru/upload/iblock/b69/b6979c7...,Максимова Вероника,NaN,5.0,Коты-воители,Твердая бумажная,7000.0,20.7 x 13 x 1.8,Насыров Леонид,304.0
1,1068930,2667992,9785171095802,Фрай М.,18+,2018,0,АСТ,768.0,Все сказки старого Вильнюса. Начало,https://img-gorod.ru/upload/iblock/d28/d28c850...,NaN,NaN,0.0,Макс Фрай: подарочное издание,Твердая бумажная,3000.0,24 x 17 x 5,NaN,1017.0
250,1077060,2678300,9785040899456,Емец Д.,12+,2018,1,Эксмо,432.0,Танец меча,https://img-gorod.ru/upload/iblock/0f0/0f0fe10...,NaN,NaN,5.0,Мефодий Буслаев. Легендарное детское фэнтези,Твердая бумажная,2000.0,21.8 x 14.4 x 2.3,NaN,342.0
155,1077461,2677963,9785001115052,Хантер Э.,6+,2018,3,Абрис Олма,382.0,Нерассказанные истории. Месть Кленовницы,https://img-gorod.ru/upload/iblock/1af/1afe28c...,Максимова В.,NaN,5.0,Коты-воители,Твердая бумажная,7000.0,20.6 x 13 x 2,Насыров Л.,304.0
154,1077741,2679748,9785001114178,Хантер Э.,6+,2018,1,Абрис Олма,447.0,Сумерки. Закат,https://img-gorod.ru/upload/iblock/d41/d412f4e...,Максимова В.А.,NaN,5.0,Коты-воители. Золотая коллекция,Твердая бумажная,3000.0,26.6 x 17.1 x 2.6,"Савельева Е.А., Насырова Л.Х.",585.0
0,1085970,2689500,9785171121945,Фрай М.,NaN,2018,1,АСТ,352.0,Тяжелый свет Куртейна. Синий,https://img-gorod.ru/upload/iblock/952/95251fd...,NaN,NaN,5.0,Другая сторона,NaN,NaN,NaN,NaN,374.0
153,1086610,2688299,9785001115090,Хантер Э.,6+,2019,0,Абрис,447.0,Тайна Щербатой,https://img-gorod.ru/upload/iblock/0f5/0f5e48f...,Максимова В.А.,NaN,0.0,Коты-воители. Золотая коллекция,Твердая бумажная,3000.0,26.5 x 17 x 2.4,Насыров Л.Х.,609.0
